# Introducción a LangChain

## Antes de comenzar

**Requisitos necesarios:**
1. Debe tener una cuenta en OPENAI
2. Añadir una variable de entorno con la KEY [Más información](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)
3. Instalar dependencias, por la cantidad de dependencias se recomienda ejecutar sobre un entorno virtual.

**Dependencias necesarias para este Notebook:**
1. pip install openai
2. pip install tiktoken
3. pip install langchain
4. pip install "langserve[all]"
5. pip install langchain-cli
6. pip install langchain docarray

Es posible que le de un error similar a este:
 "Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
     For further information visit ...."
Se soluciona con la siguiente instrucción: **pip3 install pydantic==1.10.9**


In [1]:
%pip install openai --quiet
%pip install tiktoken --quiet
%pip install langchain --quiet
%pip install langchain-cli --quiet
%pip install langchain docarray --quiet

#En caso de error
#%pip install pydantic==1.10.9

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List

from langchain.llms import OpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema import BaseOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.vectorstores import DocArrayInMemorySearch

## Sección 1 - Primeros pasos

### 1.1 Ejemplo sencillo de llamadas a modelos (GPT)
LangChain dispone de dos métodos para llamar a los modelos LLM como GPT: **Modo Texto** y **Modo Chat**

**Modo texto:**
Toma un texto de entrada (por ejemplo una pregunta) y devuelve una respuesta de tipo texto.

In [3]:


llm = OpenAI()
text = "Java es un lenguaje orientado a Objetos?"
result = llm.invoke(text)
print(type(result)) #el tipo devuelto es un string
print(result)

<class 'str'>


Sí, Java es un lenguaje orientado a objetos. Utiliza conceptos como clases, objetos, herencia, polimorfismo, encapsulamiento y abstracción para ayudar a los programadores a crear aplicaciones más eficientes.


**Modo Chat:**

#Modo chat: se le envía un mensaje con una lista de entradas con el formato [Rol - contenido] y devuelve una respuesta de tipo Mensaje.

En Langchain existen algunos roles predeterminandos:

- **HumanMessage:** Para un mensaje que viene de un humano/usuario.
- **AIMessage:** Para un mensaje aportado por la AI/assistant.
- **SystemMessage:** Para un mensaje del sistema
- **FunctionMessage / ToolMessage:**  Un mensaje que contiene la salida de una función, API, etc.

Tambien puede configurar un rol específico a través de la clase ChatMessage.

In [4]:
#Modo chat: se le envía un mensaje con una lista de entradas con [Rol - texto] y devuelve una respuesta de tipo Mensaje.
chat_model = ChatOpenAI()
IAText="Soy un experto en Java"
HumanText="¿Java es un lenguaje orientado a Objetos?"
messages = [AIMessage(content=IAText), 
            HumanMessage(content=HumanText)]

result = chat_model.invoke(messages)
print(type(result)) #el tipo devuelto es un IAMessage, que tiene un campo content que es un string
print(result)

<class 'langchain_core.messages.ai.AIMessage'>
content='Sí, Java es un lenguaje orientado a objetos. Esto significa que todos los elementos en Java, como clases, objetos, métodos y variables, están orientados a objetos y se basan en el concepto de programación orientada a objetos (POO). La POO se centra en la creación de objetos que tienen propiedades y comportamientos, y estos objetos interactúan entre sí para realizar tareas en un programa. Java también admite los cuatro principios fundamentales de la POO: encapsulación, herencia, polimorfismo y abstracción.'


### Sección 1.2 - Prompt templates
Normalmente  no solemos pasar mensajes(prompts) simples a los modelos LLMs. Lo más común es pasar contexto y/o instrucciones adicionales para optimizar el resultado. En causisticas donde se requieran lanzar prompts muy similares podemos usar plantillas, por ejemplo:

- **Ejemplo Prompt 1**: "como experto en Python, explicame el siguiente código:"messages = [AIMessage(content=IAText)"
- **Ejemplo Prompt 2**: "como experto en Java, explicame el siguiente código:"public ResponseEntity<String> getPetById(@PathVariable Long petId)"

Como podemos ver, este tipo de Prompt puede ser muy recurrente en un contexto de desarrollo software. Por ello, podemos construir una Template Prompt:

- **Prompt Template**: "como experto en {lenguaje}, explicame el siguiente código: {codigo}"

veamos un ejemplo:


In [5]:
#Modo chat: se le envía un mensaje con una lista de entradas con [Rol - texto] y devuelve una respuesta de tipo Mensaje.

chat_model = ChatOpenAI()
AssistantTemplate="eres un experto en {language}"
HumanTemplate="Como experto en {language}, explicame que hace el siguiente código: {codigo}"


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", AssistantTemplate),
    ("human", HumanTemplate),
])

#usamos format_messages para mapear los valores específicos en la plantilla
message = chat_prompt.format_messages(language='Python', codigo="messages = [AIMessage(content=IAText)]")
print("mensaje formateado:", message)

#invocamos el modelo con el mensaje final ya formateado y mapeado con los valores específicos: Python, la traza y el código
result = chat_model.invoke(message)

print("resultado:", result)
print("El tipo del resultado es:", type(result))

mensaje formateado: [SystemMessage(content='eres un experto en Python'), HumanMessage(content='Como experto en Python, explicame que hace el siguiente código: messages = [AIMessage(content=IAText)]')]
resultado: content='El código que has proporcionado crea una lista llamada "messages". En esta lista, se almacenan objetos de tipo "AIMessage". Cada objeto AIMessage tiene un atributo llamado "content", que es un objeto de tipo "IAText".\n\nEn resumen, el código crea una lista llamada "messages" y asigna un objeto AIMessage a cada elemento de la lista. Cada objeto AIMessage tiene un atributo "content" que contiene un objeto IAText.'
El tipo del resultado es: <class 'langchain_core.messages.ai.AIMessage'>


Como podemos ver, **el resultado no es un String**, es un tipo message de Langchain(AIMessage) con el formato: conten: respuesta. ¿y si quisera convertir la salida a string?

### Sección 1.3 - Output parsers

Otras de las capacidades de LangChain es la facilidad con la que podemos adatpar la salida el tipo de dato que necesitamos. Haciendo uso de la la clase BaseOutputParser, podemos usar parseadores predefinidos o crear uno propio.

En el siguiente ejemplo, vamos a parsear el resultado anterior con el parse predefinido StrOutputParser() para convertirlo en un String:

In [6]:
parserResult = StrOutputParser().invoke(result)
print("El tipo del resultado es:", type(parserResult))
print("El resultado es:", parserResult)

El tipo del resultado es: <class 'str'>
El resultado es: El código que has proporcionado crea una lista llamada "messages". En esta lista, se almacenan objetos de tipo "AIMessage". Cada objeto AIMessage tiene un atributo llamado "content", que es un objeto de tipo "IAText".

En resumen, el código crea una lista llamada "messages" y asigna un objeto AIMessage a cada elemento de la lista. Cada objeto AIMessage tiene un atributo "content" que contiene un objeto IAText.


En el siguiente ejemplo crearemos un parseador de salida específico para convertir un string a una lista de strings separada por ",". 

In [7]:
class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")
    
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

"""Eres un asistente útil que genera listas separadas por comas.
Un usuario pasará una categoría y deberías generar 5 objetos en esa categoría en una lista separada por comas.
SOLO devuelve una lista separada por comas, y nada más."""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "paises"})

['Argentina', 'Brasil', 'Colombia', 'México', 'Perú']

En este caso el resutado no es ni un mensaje ni un string. Es una lista de strings con el siguiente formato: *['Mexico', 'Canada', 'Brazil', 'Australia', 'France']*. Esto es porque hemos usado el parseador CommaSeparatedListOutputParser.

En este caso, para parsear el resultado se ha usado la instrucción: *chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()* pero ¿qué hace esto? ¿como funciona el operador |? resolveremos esas dudas en la siguiente sección.

## Sección 2 - (LCEL) LangChain Expression Language

El lenguaje LCEL nos permite, por ejemplo, encadenar una serie acciones en una sola línea con el comando | como vimos en la instrucción:
 
 *chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()* 
 
 Usando LCEL se reduce significativamente el número de líneas de codigo necesarias para construir una funcionalidad. En este caso, al cuando usamos el método invoke se pasa el valor de la variable text (paises) a la Prompt Template, despues se ejecuta el modelo a partir de la Prompt generada por la template, y por último se realiza el parseo de la salida del modelo a una lista separada por comas. Es decir, se ejecutan de izquierda a derecha todas las acciones dentro del Pipeline:

*chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()*

*chain.invoke({"text": "paises"})*

Veamos un ejemplo paso a paso:




In [8]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Cuentame una historia corta sobre {topic}")
chain = prompt | model
chain.invoke({"topic": "un perro"})

AIMessage(content='Había una vez un perro llamado Toby, un simpático Golden Retriever de pelaje dorado y ojos brillantes. Vivía en un pequeño pueblo rodeado de montañas y campos verdes. Toby siempre estaba lleno de energía y alegría, y su mayor pasión era explorar y descubrir nuevos lugares.\n\nUn día, mientras paseaba por el bosque, Toby escuchó un llanto proveniente de un arbusto cercano. Se acercó rápidamente y encontró a un cachorro abandonado. Sin dudarlo, Toby decidió adoptarlo y llevarlo a su hogar.\n\nEl cachorro, al que llamaron Max, pronto se convirtió en el mejor amigo de Toby. Juntos, exploraron cada rincón del pueblo, jugando y alegrando a todos los habitantes. La bondad y lealtad de Toby se hizo famosa en el pueblo, y todos lo adoraban.\n\nUn día, mientras Toby y Max jugaban cerca de un lago, vieron a una niña pequeña que estaba en peligro. Sin pensarlo dos veces, Toby se lanzó al agua y nadó hacia ella, agarrándola por el brazo y llevándola a un lugar seguro. La niña, ag

Ahora formateemos el mensaje a un string, en vez de la salida actual *AIMessage*.

In [9]:
chain = prompt | model | StrOutputParser()
chain.invoke({"topic": "un perro"})

'Había una vez un perro llamado Max, un adorable labrador dorado. Max vivía en un pequeño pueblo rodeado de hermosos campos verdes.\n\nUn día, Max decidió aventurarse más allá de su casa y explorar los alrededores. Caminó por senderos desconocidos hasta que llegó a un bosque encantador. Allí, encontró a un grupo de animales que parecían perdidos y asustados.\n\nHabía un conejo tembloroso, un pájaro con una ala rota y un gato callejero. Max, con su corazón bondadoso, decidió ayudarlos. Comenzó por el conejo, guiándolo con su nariz hasta su madriguera segura. Luego, Max voló al gato en sus fuertes mandíbulas hasta un refugio donde pudiera encontrar comida y agua.\n\nFinalmente, Max se acercó al pájaro herido y lo llevó con cuidado en su boca hasta un árbol alto y seguro. Allí, el pájaro podría descansar y recuperarse sin peligro.\n\nEl perro valiente y amable regresó a su casa, pero no antes de recibir un agradecimiento especial de cada uno de los animales a los que había ayudado. Max se

Ahora, usaremos un nuevo comando Batch en vez de invoke. En este caso para ejecutar el pipeline para varias entradas:

In [10]:
chain.batch([{"topic": "Un perro"}, {"topic": "Un gato"}])

['Había una vez un perro llamado Max, un labrador dorado muy juguetón y cariñoso. Vivía en un pequeño pueblo junto a su dueño, Lucas, quien lo quería como a un miembro más de su familia.\n\nUn día, Lucas decidió llevar a Max de paseo por el bosque cercano al pueblo. Era un lugar hermoso, lleno de árboles altos y flores de colores. Max estaba emocionado, moviendo su cola de un lado a otro mientras exploraban el lugar.\n\nDe repente, escucharon un ruido extraño proveniente de unos arbustos. Max, curioso como siempre, corrió hacia allí. Al acercarse, encontraron a una pequeña cachorra perdida. Tenía el pelaje blanco y estaba temblando de miedo.\n\nLucas y Max la acogieron de inmediato. Decidieron llevarla a casa, donde le dieron un baño y un poco de comida. La llamaron Luna, por su suave pelaje blanco que parecía reflejar la luz de la luna.\n\nMax se convirtió en el mejor amigo y protector de Luna. Juntos, exploraban el pueblo y jugaban sin cesar. Luna había encontrado una familia que la 

En esta ocasion, con el comando batch se ha ejecutado 2 veces el proceso. En el siguiente ejemplo usaremos stream, que nos permitirá imprimir el resultado palabra a palabra a medida que recibe la respuesta en vez de esperar a tener toda la respuesta.

In [11]:
for s in chain.stream({"topic": "un perro"}):
    print(s, end="", flush=True)

Había una vez un perro llamado Max, que vivía en un pequeño pueblo junto a su dueño, Juan. Max era un perro muy juguetón y le encantaba salir a pasear por el parque todos los días.

Un día, mientras Max y Juan caminaban por el parque, escucharon un llanto proveniente del fondo de un arbusto. Max, moviendo su cola emocionado, se acercó rápidamente para investigar. Allí encontraron a un cachorro abandonado y asustado.

Juan y Max decidieron llevar al cachorro a casa y cuidarlo. Le pusieron el nombre de Toby y se convirtió en el nuevo miembro de la familia. Max y Toby se volvieron inseparables y pasaban todo el día jugando y explorando juntos.

Con el tiempo, Max notó que Toby tenía miedo de los ruidos fuertes, como los truenos. Max, recordando cómo Juan lo había ayudado a superar su propio miedo a los fuegos artificiales, decidió enseñarle a Toby a no tener miedo.

Max comenzó a llevar a Toby a un lugar tranquilo en el parque cuando escuchaban truenos o fuegos artificiales, para mostrarl

## Sección 3 - Jugando con Embeddings y Vectors DBs(In progress)

En el siguiente ejemplo se muestra un ejemplo de RAG sensillo donde se almacenan los embeddings en memoria.

In [12]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Proceso 1: Los empleados deben imputar sus horas en cada una de las tareas en las que ha trabajado cada día", 
     "Proceso 2: En caso de que el empleado no haya trabajado en una tarea, debe imputar las horas en la tarea 'Otros'",
     "Proceso 3: Las vacaciones se imputarán en la tarea 'Vacaciones', pero debe enviarse un mail a RRHH para que se contabilicen correctamente",
     "Proceso 4: Las horas extras se imputarán en la tarea 'Horas extras', pero debe enviarse un mail a RRHH para que se contabilicen correctamente",
     "Proceso 5: Las horas de formación se imputarán en la tarea 'Formación', el empleado debe firmar un documento de asistencia y enviarlo a RRHH",
     "Norma 1: El empleado debe imputar sus horas en el sistema antes de las 18:00 del día siguiente",
     "Norma 2: Las vacaciones deben tomarse para un perido mínimo de 5 días naturales"],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """Responde a la pregunta teniendo en cuenta en primer lugar el siguiente contexto sin ser muy creativo:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

print(chain.invoke("Hoy he asistido a una formación de 2H, y el resto del tiempo he trabajado de tareas. ¿cual es el procedimiento?"))

El procedimiento sería el siguiente:
1. Para las horas de formación, debes imputarlas en la tarea "Formación".
2. Además, debes firmar un documento de asistencia y enviarlo a RRHH.
3. Para el resto del tiempo que has trabajado en otras tareas, debes imputar las horas en cada una de esas tareas correspondientes.
4. En caso de que no hayas trabajado en una tarea específica, debes imputar las horas en la tarea "Otros".
5. Si has tomado vacaciones, debes imputarlas en la tarea "Vacaciones" y también enviar un correo a RRHH para que se contabilicen correctamente.


Base de datos vectorial